# Your first nonlinear solver- newton-raphson iteration

- [x] newton-raphson method
- [x] weak form
- [x] residual and jacobian
- [x] nonlinear iteration
- [x] convergence criterion
- [x] nonlinear poisson equation

Author: Yang Bai @ M3 Group

Date  : 2022.04.17

QQ group: 628204857

# Newton-raphson method

How to find the root of $f(x)=0$?

Find $x_{0}$ which s.a. $f(x_{0})=0$.

## Taylor expansion
$f(x^{})=f(x^{k})+\frac{\partial f(x^{k})}{\partial x}(x^{}-x^{k})+\frac{\partial^{2} f(x^{k})}{\partial x^{2}}\frac{1}{2!}(x^{}-x^{k})^{2}+\dots$

$f(x_{0})=f(x^{k})+\frac{\partial f(x^{k})}{\partial x}(x_{0}-x^{k})=0$

$x_{0}=x^{k}-\frac{f(x^{k})}{\frac{\partial f(x^{k})}{\partial x}}$

$f(x^{k})=-\frac{\partial f(x^{k})}{\partial x}(x_{0}-x^{k})=K\Delta x$, with $K=-\frac{\partial f(x^{k})}{\partial x}$, $\Delta x=x_{0}-x^{k}$

## Example

### Ex. 1 
$f(x)=x-1=0$

$K=-\frac{\partial f}{\partial x}=-1$

1st iteration: set x=0, f(x)=-1 ===> dx=K\f=1

--- update x=x+dx=1  check f(x)=0, done!

### Ex 2
$f(x)=1.0+sin(x)-x$

$K=-\frac{\partial f}{\partial x}=-cos(x)+1$

In [1]:
import numpy as np
def f(x):
    return 1.0+np.sin(x)-x
def df(x):
    return np.cos(x)-1

x=0.5;rnorm=1.0;iters=0;tolerance=1.0e-10
while iters<200 and rnorm>tolerance:
    R=f(x);K=-df(x)
    dx=R/K
    x+=dx
    rnorm=np.abs(R)
    iters+=1
    print('iters=%3d, |R|=%14.6e, |dx|=%14.6e'%(iters,rnorm,np.abs(dx)))
if(not rnorm>tolerance):
    print('the converged solution is: x=%14.6e, f(x)=%15.8e'%(x,f(x)))
else:
    print('oops, your newton-raphson failed!')

iters=  1, |R|=  9.794255e-01, |dx|=  8.000703e+00
iters=  2, |R|=  6.702639e+00, |dx|=  4.182424e+00
iters=  3, |R|=  4.241618e+00, |dx|=  3.064783e+00
iters=  4, |R|=  6.965849e-01, |dx|=  1.012482e+00
iters=  5, |R|=  4.980408e-01, |dx|=  3.035862e-01
iters=  6, |R|=  3.809059e-02, |dx|=  2.756864e-02
iters=  7, |R|=  3.525588e-04, |dx|=  2.599914e-04
iters=  8, |R|=  3.158407e-08, |dx|=  2.329557e-08
iters=  9, |R|=  4.440892e-16, |dx|=  3.275484e-16
the converged solution is: x=  1.934563e+00, f(x)=-2.22044605e-16


# Newton-raphson in FEM

For a FEM problem $\Pi\sim\Pi(u,\nabla u,\dot{u},\dots)$, one need to find $u$ s.a. $\delta\Pi=R(u,\nabla u,\dot{u},\dots)\delta u=0$ with $R=[K(u,\nabla u,\dot{u})]\{u\}-f(u,\nabla u,\dot{u})$

## Nonlinear poisson equation

$\nabla\cdot(\sigma\nabla\phi)=f$ with $\sigma\sim\sigma(\phi)$ and $f\sim f(\phi)$, $-\sigma\nabla\phi\cdot\vec{n}=0$

### weak form
$\int_{\partial\Omega}\sigma\nabla\phi\cdot\vec{n}\delta\phi dS-\int_{\Omega}\sigma\nabla\phi\nabla\delta\phi dV=\int_{\Omega}f\delta\phi dV$

then the residual and jacobian matrix can be read as follows:

$R^{I}=\int_{\Omega}\sigma\nabla\phi\nabla N^{I}dV+\int_{\Omega}fN^{I}dV$

$K^{IJ}=-\frac{\partial R^{I}}{\partial\phi^{J}}=-\int_{\Omega}\frac{\partial\sigma}{\partial\phi}N^{J}\nabla\phi\nabla N^{I}dV-\int_{\Omega}\sigma\nabla N^{J}\nabla N^{I}dV-\int_{\Omega}\frac{\partial f}{\partial\phi}N^{J}N^{I}dV$

In [2]:
from FEToy.mesh.lagrange2dmesh import mesh2d
from FEToy.fe.shapefun import shape2d
from FEToy.fe.gaussrule import gausspoint2d
from FEToy.postprocess.Result import ResultIO

# define your mesh
mymesh=mesh2d(xmax=2.0,ymax=2.0,nx=20,ny=20,meshtype='quad4')
mymesh.createmesh()

# define your shape function
shp=shape2d(meshtype='quad4')
shp.update()

# define your gauss points
qpoints=gausspoint2d(ngp=3)
qpoints.creategausspoint()


### defines your physical parameters here
# since sigma and f both are nonlinear, we can offer functions for them
def sigma(x):
    return 1+np.exp(x)
    # return 1.0+x*x
def dsigma(x):
    return np.exp(x)
    # return 2*x
### for f
def f(x):
    return 1.0+0.1*x
def df(x):
    return 0.1

# define your K and rhs, and other vectors
nDofs=mymesh.nodes*1
K=np.zeros((nDofs,nDofs))
R=np.zeros(nDofs)
U=np.zeros(nDofs)
penalty=1.0e16 # for dirichlet bc

# for parameters used in NR iteration
tolerance=1.0e-10;reltol=1.0e-15 # absolute tolerenace and relative tolerance
maxiters=50
rnorm=1.0;rnorm0=1.0;iters=0 # init
dunorm=1.0;dunorm0=1.0
enorm=1.0;enorm0=1.0
converged=False

gradphi=np.zeros(2)
# now we can do the newton-raphson iteration
while iters<maxiters and (not converged):
    K[:,:]=0.0;R[:]=0.0 # init
    
    # do the element loop
    for e in range(mymesh.elements):
        elconn=mymesh.elementconn[e,:]
        nodes=mymesh.nodecoords[elconn]
        for gp in range(qpoints.ngp2):
            xi =qpoints.gpcoords[gp,1]
            eta=qpoints.gpcoords[gp,2]
            w  =qpoints.gpcoords[gp,0]
            shp_val,shp_grad,j=shp.calc(xi,eta,nodes[:,0],nodes[:,1]) # xi,eta,x,y
            JxW=j*w # the weight times the determinate of jacobian transformation
            
            phi=0.0 # for the phi on current gauss point
            gradphi[0]=0.0;gradphi[1]=0.0
            x=0.0;y=0.0 # coordinates of current gauss point
            for i in range(mymesh.nodesperelement):
                iInd=elconn[i] # global id
                phi+=shp_val[i]*U[iInd]
                gradphi[0]+=shp_grad[i,0]*U[iInd]
                gradphi[1]+=shp_grad[i,1]*U[iInd]
                x+=shp_val[i]*nodes[i,0]
                y+=shp_val[i]*nodes[i,1]
            scale=1.0+np.sin(x*y)
            # scale=1.0
            for i in range(mymesh.nodesperelement):
                iInd=elconn[i]
                # assemble to global F
                R[iInd]+=scale*sigma(phi)*(gradphi[0]*shp_grad[i,0]+gradphi[1]*shp_grad[i,1])*JxW\
                        +f(phi)*shp_val[i]*JxW
                # now we can calculate the K matrix
                for j in range(mymesh.nodesperelement):
                    jInd=elconn[j]
                    K[iInd,jInd]+=(-1.0)*scale*dsigma(phi)*shp_val[j]*(gradphi[0]*shp_grad[i,0]+gradphi[1]*shp_grad[i,1])*JxW\
                                 +(-1.0)*scale*sigma(phi)*(shp_grad[j,0]*shp_grad[i,0]+shp_grad[j,1]*shp_grad[i,1])*JxW\
                                 +(-1.0)*df(phi)*shp_val[j]*shp_val[i]*JxW
    ### end-of-element-loop
    
    ###################################################################
    ### now we apply the boundary condition
    ### for dirichlet bc, please keep in mind, you are within a
    ### NR iteration, your dx should remain zero when the 
    ### dof is a 'dirichlet boundary dof'(because x=x+dx) !
    ###################################################################
    # here we fix all the boundary 'phi' to a constant value
    iInd=mymesh.bcnodeids['left']
    K[iInd,iInd]+=penalty
    R[iInd]=0.0
    # iInd=mymesh.bcnodeids['right']
    # K[iInd,iInd]+=penalty
    # R[iInd]=0.0
    # iInd=mymesh.bcnodeids['bottom']
    # K[iInd,iInd]+=penalty
    # R[iInd]=0.0
    # iInd=mymesh.bcnodeids['top']
    # K[iInd,iInd]+=penalty
    # R[iInd]=0.0
    
    # now all your K and R are assembled, you can solve them
    dU=np.linalg.solve(K,R)
    
    # update info
    iters+=1
    U+=dU
    rnorm=np.linalg.norm(R)
    dunorm=np.linalg.norm(dU)
    enorm=rnorm*dunorm
    if iters==1:
        rnorm0=rnorm
        dunorm0=dunorm
        enorm0=enorm
    print('iters=%3d, |R0 |=%14.5e, |R |=%14.5e'%(iters,rnorm0,rnorm))
    print('           |dU0|=%14.5e, |dU|=%14.5e'%(dunorm0,dunorm))
    print('           |E0 |=%14.5e, |E |=%14.5e'%(enorm0,enorm))
    
    # convergence check
    if rnorm<tolerance or rnorm<reltol*rnorm0:
        converged=True
        break
### end-of-NR-loop
result=ResultIO('nlpoisson')
result.save2vtu(mesh=mymesh,solution=U,varnamelist=['phi'],step=0)

iters=  1, |R0 |=   1.93746e-01, |R |=   1.93746e-01
           |dU0|=   1.05849e+01, |dU|=   1.05849e+01
           |E0 |=   2.05079e+00, |E |=   2.05079e+00
iters=  2, |R0 |=   1.93746e-01, |R |=   4.72237e-02
           |dU0|=   1.05849e+01, |dU|=   1.55848e+00
           |E0 |=   2.05079e+00, |E |=   7.35972e-02
iters=  3, |R0 |=   1.93746e-01, |R |=   9.57083e-04
           |dU0|=   1.05849e+01, |dU|=   2.46349e-02
           |E0 |=   2.05079e+00, |E |=   2.35776e-05
iters=  4, |R0 |=   1.93746e-01, |R |=   4.03722e-07
           |dU0|=   1.05849e+01, |dU|=   6.51174e-06
           |E0 |=   2.05079e+00, |E |=   2.62893e-12
iters=  5, |R0 |=   1.93746e-01, |R |=   5.72883e-14
           |dU0|=   1.05849e+01, |dU|=   5.33811e-13
           |E0 |=   2.05079e+00, |E |=   3.05811e-26
write result to nlpoisson-000000.vtu
